In [3]:
import mitsuba as mi
mi.variants()
mi.set_variant("scalar_spectral")
scene = mi.load_file("lego/cbox.xml")

from mitsuba import ScalarTransform4f as T

def load_sensor(r, phi, theta):
    # Apply two rotations to convert from spherical coordinates to world 3D coordinates.
    origin = T.translate([0, -1.2, 0]) @ T.rotate([0, 1, 0], phi).rotate([1, 0, 0], theta) @ mi.ScalarPoint3f([0, 0, r])

    return mi.load_dict({
        'type': 'perspective',
        'fov': 39.37,
        'to_world': T.look_at(
            origin=origin,
            target=[0, -3.7, 0],
            up=[0, 1, 0]
        ),
        'sampler': {
            'type': 'independent',
        },
        'film': {
            'type': 'specfilm',
            'width': 2000,
            'height': 2000,
            'band4': {
                'type': 'spectrum',
                'value': [(548.0, 0.01), (557.0, 0.27), (567.0, 0.54), (577.0, 0.27), (587.0, 0.01)]
            },
            'band5': {
                'type': 'spectrum',
                'value': [(593.0, 0.01), (602.0, 0.275), (611.0, 0.55), (620.0, 0.275), (629.0, 0.01)]
            },
            'band6': {
                'type': 'spectrum',
                'value': [(649.0, 0.01), (657.0, 0.28), (666.0, 0.56), (675.0, 0.28), (683.0, 0.01)]
            }
        },
    })

radius = 2
phis = [20.0*(i+1) for i in range(10)]
thetas = [20.0*(j+1) for j in range(10)]

sensors = []
for phi in phis:
        for theta in thetas:
            sensors.append(load_sensor(radius,theta,phi))
sensor_count = len(sensors)

images = [mi.render(scene, spp=512, sensor=sensor) for sensor in sensors]


# Saving the rendered Images. 
import os
folder_name = "rendered_images_rgb2"

# Create the folder if it doesn't exist
os.makedirs(folder_name, exist_ok=True)

# Save the rendered images with descriptive filenames
for i, image in enumerate(images):
    filename = f"{folder_name}/view_{i+1:04d}.png"
    mi.util.write_bitmap(filename, image)
    
print(f"Rendered images saved to: {folder_name}")

Rendered images saved to: rendered_images_rgb2
